In [35]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from pypinyin import lazy_pinyin,Style
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, auc, roc_auc_score
import joblib

In [17]:
lazy_pinyin('小纪最帅')

['xiao', 'ji', 'zui', 'shuai']

In [2]:
data=pd.read_csv('text_all.csv')

In [4]:
data.head()

,text,label
0,颙 29526 Q 77544,1
1,﻿染-黄色 K U C 5 3 4,1
2,91网址求大哥,1
3,塞 CC-名字-看拼ᚰ,1
4,QQ网名！微信,0


https://blog.csdn.net/Daisy4/article/details/121548990 df.sample 的解释

In [5]:
data=data.sample(frac=1,random_state=42) # frac 为抽取比例，random_state 为随机种子

In [6]:
data.head()

,text,label
17618,我要看靓仔的猛男♂时刻,0
7664,美国自驾游,0
18679,骗吃骗喝 美滋滋,0
8054,卖片卖片的,1
2495,8695 33334 粉逼加口,1


In [7]:
data['label'].value_counts()

1    9882
0    9788
Name: label, dtype: int64

In [8]:
data['text']=data['text'].apply(lambda x: ' '.join(x))

In [9]:
data.head()

,text,label
17618,我 要 看 靓 仔 的 猛 男 ♂ 时 刻,0
7664,美 国 自 驾 游,0
18679,骗 吃 骗 喝 美 滋 滋,0
8054,卖 片 卖 片 的,1
2495,8 6 9 5 3 3 3 3 4 粉 逼 加 口,1


In [13]:
x_train,x_test,y_train,y_test=train_test_split(
data['text'],data['label'],test_size=0.2,random_state=42)

In [14]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(15736,) (3934,) (15736,) (3934,)


In [22]:
vectorizer_word=TfidfVectorizer(
    max_features=800000,                              
    token_pattern=r"(?u)\b\w+\b",                                   
    min_df=5,                              
#max_df=0.1,                               
    analyzer='word',                               
    ngram_range=(2,5) 
)

In [23]:
vectorizer_word.fit(x_train) 
tfidf_word_train = vectorizer_word.transform(x_train) 
tfidf_word_test  = vectorizer_word.transform(x_test)

In [55]:
print(len(vectorizer_word.vocabulary_)) #词袋

42429


17.token_pattern: "(?u)\b\w+\b"
解析：
[1](?u)：对Unicode符大小写不敏感。
[2]\b：匹配单词的开始或结束。
[3]\w：匹配字母或数字或下划线或汉字。
[4].：匹配除换行符以外的任意字符。
[5]\s：匹配任意的空白符。
[6]\d：匹配数字。
[7]\w+：匹配一个或者多个字母或数字或下划线或汉字。
[8](?u)\b\w+\b：这个正则表达式会忽略掉单个的字符。
[9](?u)\b\w\w+\b：这个正则表达式不会忽略掉单个的字符。
说明：说明；\b代表字与字中间那个看不见的东西，比如here is a word。那么这句中有好几个\b，每个单词的前后都有一个\b。

def init_params(label='TfidfVectorizer'):
    params_count={
        'analyzer': 'word',  # 取值'word'-分词结果为词级、'char'-字符级(结果会出现he is，空格在中间的情况)、'char_wb'-字符级(以单词为边界)，默认值为'word'
        'binary': False,  # boolean类型，设置为True，则所有非零计数都设置为1.（即，tf的值只有0和1，表示出现和不出现）
        'decode_error': 'strict',
        'dtype': np.float64, # 输出矩阵的数值类型
        'encoding': 'utf-8',
        'input': 'content', # 取值filename，文本内容所在的文件名；file，序列项必须有一个'read'方法，被调用来获取内存中的字节；content，直接输入文本字符串
        'lowercase': True, # boolean类型，计算之前是否将所有字符转换为小写。
        'max_df': 1.0, # 词汇表中忽略文档频率高于该值的词；取值在[0,1]之间的小数时表示文档频率的阈值，取值为整数时(>1)表示文档频数的阈值；如果设置了vocabulary，则忽略此参数。
        'min_df': 1, # 词汇表中忽略文档频率低于该值的词；取值在[0,1]之间的小数时表示文档频率的阈值，取值为整数时(>1)表示文档频数的阈值；如果设置了vocabulary，则忽略此参数。
        'max_features': None, # int或 None(默认值).设置int值时建立一个词汇表，仅用词频排序的前max_features个词创建语料库；如果设置了vocabulary，则忽略此参数。
        'ngram_range': (1, 2),  # 要提取的n-grams中n值范围的下限和上限，min_n <= n <= max_n。
        'preprocessor': None, # 覆盖预处理（字符串转换）阶段，同时保留标记化和 n-gram 生成步骤。仅适用于analyzer不可调用的情况。
        'stop_words': 'english', # 仅适用于analyzer='word'。取值english，使用内置的英语停用词表；list，自行设置停停用词列表；默认值None，不会处理停用词
        'strip_accents': None,
        'token_pattern': '(?u)\\b\\w\\w+\\b', # 分词方式、正则表达式，默认筛选长度>=2的字母和数字混合字符（标点符号被当作分隔符）。仅在analyzer='word'时使用。
        'tokenizer': None, # 覆盖字符串标记化步骤，同时保留预处理和 n-gram 生成步骤。仅适用于analyzer='word'
        'vocabulary': None, # 自行设置词汇表（可设置字典），如果没有给出，则从输入文件/文本中确定词汇表
    }
    params_tfidf={
        'norm': None, # 输出结果是否标准化/归一化。l2：向量元素的平方和为1，当应用l2范数时，两个向量之间的余弦相似度是它们的点积；l1：向量元素的绝对值之和为1
        'smooth_idf': True, # 在文档频率上加1来平滑 idf ，避免分母为0
        'sublinear_tf': False, # 应用次线性 tf 缩放，即将 tf 替换为 1 + log(tf)
        'use_idf': True, # 是否计算idf，布尔值，False时idf=1。
    }
    if label=='CountVectorizer':
        return params_count
    elif label=='TfidfTransformer':
        return params_tfidf
    elif label=='TfidfVectorizer':
        params_count.update(params_tfidf)
        return params_count

In [26]:
data=pd.read_csv('text_all.csv')
data['text']=data['text'].apply(lambda x: ' '.join(lazy_pinyin(x)))
data.head()

,text,label
0,yong 29526 Q 77544,1
1,﻿ ran - huang se K U C 5 3 4,1
2,91 wang zhi qiu da ge,1
3,sai CC- ming zi - kan pin ᚰ,1
4,QQ wang ming ！ wei xin,0


In [28]:
x_train,x_test,y_train,y_test=train_test_split(
data['text'],data['label'],test_size=0.2,random_state=42)

In [29]:
vectorizer_word = TfidfVectorizer(max_features=800000,
                             min_df = 5, 
                             analyzer='char', 
                             ngram_range=(1,7)
                             )

vectorizer_word.fit(x_train)

tfidf_pinyin_train = vectorizer_word.transform(x_train)
tfidf_pinyin_test  = vectorizer_word.transform(x_test)

In [31]:
print(len(vectorizer_word.vocabulary_)) #查看词袋

42429


In [33]:
import scipy.sparse as sp #组合向量
tfidf_all_train = sp.hstack((tfidf_word_train,tfidf_pinyin_train))
tfidf_all_test  = sp.hstack((tfidf_word_test,tfidf_pinyin_test))

In [48]:
lr_word = LogisticRegression(solver='sag',  #梯度下降
                             verbose=2,
                             C=5.0 #正则化强度，倒数。
                             )
lr_word.fit(tfidf_all_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 34 epochs took 1 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished


LogisticRegression(C=5.0, solver='sag', verbose=2)

In [49]:
y_pred_word = lr_word.predict(tfidf_all_test)

In [50]:
print(accuracy_score(y_test, y_pred_word))

0.9580579562785968


In [52]:
# 预测错误的数值
data = pd.read_csv('text_all.csv')
x_train, x_test, y_train, y_test = train_test_split(
                     data['text'], 
                     data['label'], 
                     test_size=0.2, 
                     random_state=42
                     )
df = pd.DataFrame({'text':x_test,'label':y_test,'pred':y_pred_word})
df[df['label']!=df['pred']]

,text,label,pred
6126,__𝐂𝐃１６７,1,0
12636,mlc2747141543,1,0
4084,加我VX----a13713637668,1,0
19364,看名字诚心卖,1,0
14059,扣扣937787511,0,1
...,...,...,...
3100,泡泡买看好 ZZ1600的,1,0
8550,篇国找C3003D,1,0
12954,网址？,0,1
6885,D.国.著.名.影.星.名.字,1,0


In [59]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.sans-serif']=['KaiTi']
plt.rcParams['axes.unicode_minus']=False

In [ ]:

pos_df = pd.DataFrame([line.replace('\n', '') for line in open('positive.txt',encoding='utf-8').readlines()],columns=['text'])
pos_df['label']=1    #正面标签为1
neg_df = pd.DataFrame([line.replace('\n', '') for line in open('negtive.txt',encoding='utf-8').readlines()],columns=['text'])
neg_df['label']=0 